# Pull Request n.2 (#5212)

In [1]:
import pennylane as qml

Lo scopo di questa PR is quello di rimpiazzare l'uso di `map_batch_transform` nel codice sorgente.
La TZ vuole dichiarare questa funzione come deprecata in futuro, quindi tanto vale rimpiazzare il current usage. 

Al momento la funzione si trova nei seguenti punti:

- ~~https://github.com/PennyLaneAI/pennylane/blob/4542986500c41a1c77022bc7728ccb26dc4b938d/pennylane/_device.py#L806~~  
- ~~https://github.com/PennyLaneAI/pennylane/blob/4542986500c41a1c77022bc7728ccb26dc4b938d/pennylane/workflow/jacobian_products.py#L314~~ 
- ~~https://github.com/PennyLaneAI/pennylane/blob/4542986500c41a1c77022bc7728ccb26dc4b938d/pennylane/workflow/jacobian_products.py#L331~~
- ~~https://github.com/PennyLaneAI/pennylane/blob/4542986500c41a1c77022bc7728ccb26dc4b938d/pennylane/workflow/execution.py#L200~~

La TZ scrive anche:

**Note that a partial was used together with `map_batch_transform`, as `map_batch_transform` didn't accept args and kwargs.**

(...)

**But we would need to turn `dev_batch_transform` into a `qml.transform` before applying it to the batch.**

To replace the usage in `pennylane/workflow/jacobian_products.py#L314` sto utilizzando il test `tests/interfaces/test_jacobian_products.py`, in cui la funzione viene di fatto chiamata.

Sono riuscito ad estrapolare il code snippet in cui tale funzione viene chiamata, in questo modo posso testarla meglio.

In [2]:
from pennylane.workflow.jacobian_products import TransformJacobianProducts

dev = qml.device("default.qubit")


def inner_execute_numpy(tapes):
    return dev.execute(tapes)


jpc = TransformJacobianProducts(inner_execute_numpy, qml.gradients.param_shift)

x = 1.62
tape = qml.tape.QuantumScript([qml.RX(x, 0)], [qml.expval(qml.PauliZ(0))], shots=10000)
results, jacs = jpc.execute_and_compute_jacobian((tape,))

jac = jpc.compute_jacobian((tape,))

Con un po' piu di fatica sono riuscito ad estrapolare da `tests/circuit_graph/test_circuit_graph.py` un code snippet che esegue la `map_batch_transform` da rimpiazzare in `pennylane/workflow/execution.py#L200`

In [4]:
from pennylane import numpy as pnp


def parameterized_circuit_gaussian_fixture(wires):
    def qfunc(a, b, c, d, e, f):
        qml.Rotation(a, wires=wires[0])
        qml.Rotation(b, wires=wires[1])
        qml.Rotation(c, wires=wires[2])
        qml.Beamsplitter(d, 1, wires=[wires[0], wires[1]])
        qml.Rotation(1, wires=wires[0])
        qml.Rotation(e, wires=wires[1])
        qml.Rotation(f, wires=wires[2])

        return qml.expval(qml.ops.NumberOperator(wires=wires[0]))

    return qfunc


wires = ["a", "q1", 3]

dev = qml.device("default.gaussian", wires=wires)
qnode = qml.QNode(parameterized_circuit_gaussian_fixture(wires), dev)


qnode(*pnp.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6], requires_grad=True))


circuit = qnode.qtape.graph
layers = circuit.parametrized_layers
ops = circuit.operations